### This personal notebook describes notes about aws and cli
In this sample we will build in central (frankfurt) region our pred-reclaim api server.

#### Install CLI

```
Windows: https://s3.amazonaws.com/aws-cli/AWSCLI64.msi

Linux or mac requires pip to install, which requires setuptools itself. If you have pip just install the awscli)

SETUPTOOLS:
wget https://pypi.python.org/packages/source/s/setuptools/
setuptools-7.0.tar.gz
tar xvf setuptools-7.0.tar.gz
cd setuptools-7.0
python setup.py install

PIP:
wget https://bootstrap.pypa.io/get-pip.py
python get-pip.py

AWSCLI

pip install awscli

TEST:
aws -version



```
    
    

### Access and security

As we use an existing organisation for now we will not create any users and groups, maybe roles later for you api

```
Configure:

Start configure screen, if you have multiple users using aws cli 
, use different profiles.

root@master:/setuptools-7.0# aws configure (--profile <somename>)
AWS Access Key ID [None]: <secret>
AWS Secret Access Key [None]: <secret>
Default region name [None]: eu-central-1 (or eu-west-1)
Default output format [None]: table

users:
aws iam list-users
aws iam create-user --user-name <user>
aws iam create-login-profile --user-name <user> --password <pwd>
aws iam create-access-key --user <username>

groups:
aws iam create-group --group-name <grp>
aws iam add-user-to-group --user-name <user> --group-name <user>

profiles:
create this file:
{
"Version": "2012-10-17",
"Statement": [
      {
"Effect": "Allow",
"Action": "*",
"Resource": "*"
} ]
}

Grant to user (can also be group):
aws iam put-group-policy --user-name <user> \
--policy-name <all privs for your your org> \
--policy-document file:///tmp/<your file>.json

``` 

### EC2 instances

#### First some general notes

```
ec2 is container running ami images:

two types ami:
Instance store-backed instance via S# (slow)
ebs (elastic block store), shared drive, quick portable, peristant storage, also can handle 16tb instead of 10gb.  but does cost some extra

types:
memory, computer, storage or general optimised, or lastly gpu for graphical/machine learning

pricing:
on-demand , per hour, no cost when not using but higher price
reserved: cheaper on the whole, but you do pay when not using. yu can pay upfront (partly) making it cheaper.
spot: bid on capacity, no guarantee at all.

``` 

### Install server

``` 
in our keys we will use dil-pred-reclaim-key-pair as name

public key:
aws ec2 create-key-pair --key-name dil-pred-reclaim-key-pair \
 --output text > dil-pred-reclaim-key-pair.pem

chmod 400 dil-pred-reclaim-key-pair.pem

security group:
aws ec2 create-security-group --group-name sg_dil-pred-reclaim \
 --description "Security group predictive reclaim api server"

Rules:
aws ec2 authorize-security-group-ingress \
--group-name sg_dil-pred-reclaim \
--protocol tcp --port 22 --cidr 0.0.0.0/0

run (aws linux):
aws ec2 run-instances --image-id ami-b968bad6 \
 --count 1 --instance-type t2.small \
 --security-groups sg_dil-pred-reclaim \
 --key-name dil-pred-reclaim-key-pair \
 --tag-specifications 'ResourceType=instance,Tags=[{Key=Name,Value=dil-pred-reclaim}]'

test (does not work yet, pem has to be altered?):
ssh -i dil-pred-reclaim-key-pair.pem ec2-user@<PUBLIC ip>

``` 